# Cotejamiento de Cedulas Hospital General Milagro

### A continuacion se realiza el cotejamiento de las cedulas de trabajadores y ex-trabajadores de la Empresa los Alamos segun Solicitud en Memorando Nro.IESS-CPPCG-2023-2144-A, contra los archivos planos del Hospital General Milagro

***

#### Instalamos las librerias necesarias para procesar cantidades masivas de datos

In [ ]:
!pip install -q pandas

#### Importamos las librerias necesarias para los siguientes pasos

In [ ]:
import os
import pandas as pd
import warnings

#### Desactivamos los warnings(alertas) sobre tamaños masivos de datos, etc

In [ ]:
warnings.filterwarnings("ignore")

#### Obtenemos he imprimimos la direccion de la carpeta actual de este notebook, que a su vez sera la direccion donde se va ha guardar todos los datos generados

In [ ]:
path = os.getcwd()
columnas = ['DEPENDENCIA', 'CODIGO_ATENCION', 'FECHA_ATENCION', 'TIPO_DE_AFILIACION', 'CEDULA', 'NOMBRES', 'SEXO',
            'FECHA_NACIMIENTO', 'EDAD', 'PROCEDIMIENTO', 'CODIGO_PROCEDI', 'DESCRIPCION', 'CIE_10', 'CIE10_2',
            'CIE10_3', 'CANTIDAD', 'VALOR_UNITARIO', 'DURACION', 'PARENTESCO', 'IDENTIFICACION_AFILIADO',
            'NOMBRES_AFILIADO', 'CODIGO_DERIVACION', 'DERIVACION_SECUENCIAL', 'CONTINGENCIA_CUBIERTA',
            'DIAGNOSTICO_PRESUNTIVO_DEFINITIVO', 'TIEMPO_DE_ANESTESIA', 'VALOR_%_IVA', 'VALOR_UNITARIO_IVA',
            'CODIGO_DE_LA_UNIDAD', 'CODIGO_PROFESIONAL', 'NOMBRE_PROFESIONAL', 'MARCA_FINAL_DE_LA_LINEA']
print(path)

#### El nombre del DataFrame que vamos a utilizar puede varias dependiendo de los criterios que se determinen, en este caso se le puso como nombre "alamos"
#### Cargamos el archivo de Excel donde se encuentran las cedulas de los trabajadores de la Empresa los Alamos, por motivos practicos el nombre se lo dejo como el Numero de Memorando en el cual fue remitido el listado, el segundo parametro "1" es el nombre que tiene la hoja del libro de excel donde vamos a trabajar, Pandas obliga o colocar el nombre de la hoja en la que se va ha trajar en caso de que se carge un archivo de Excel con multiples hojas, a continuacion una recomendacion de los nombres para las columnas del archivo de excel par evitar errores por espacios o signos especiales:
- ORD
- CEDULA
- NOMBRES_APELLIDOS
- PERIODO
- MEMORANDO_NRO
- FECHA_PRESTACIONES_MEDICAS1 
- DEPENDENCIA_SERVICIOS
- VALOR_TOTAL
- REGISTRA_SUBSIDIOS
- DESDE
- HASTA
- VALOR_PRESTACIONES
- OBSERVACIONES
##### Nos Aseguramos de que la Columna 'CEDULA' este formateada en tipo texto con 10 caracteres y si no es asi ponga el "0" al inicio del numero

In [ ]:
alamos = pd.read_excel("IESS-CPPCG-2023-2144-A.xls","1")
# alamos = alamos.drop_duplicates(subset=['CEDULA'])
alamos['CEDULA'] = alamos['CEDULA'].astype(str).str.zfill(10)
print(len(alamos))
alamos.head()

#### Como ayuda visual imprimimos los tipos de datos de cada columna de datos del DataFrame "alamos"

In [ ]:
alamos.dtypes

#### Procedemos a hacer la carga masiva de los archivos planos (csv) facilitados por factuacion, es recomendable tener los archivos planos almacenados en una carpeta por año, con un nombre homogeneo, en este caso "HG MILAGRO " seguido del año (2017,2018,2019,2020,2021,etc); los nombres de cada archivo plano debe cumplir con ciertos requisitos como no tener espacios (sustituirlos con "_") el prefijo utilizado en este caso es "HG_MI_" mas el años y el mes al que representa ese archivo plano ("HG_MI_201701",etc), el prefijo a utilizarce es entera eleccion de cada persona pero el año y mes deben ser estrictamente en el formato "yyyymm"

In [ ]:
# Crea un diccionario vacío para almacenar los datos
datos_archivo_plano = {}
subcarpetas = "HG MILAGRO"

for a in range(2017,2022):
    for archivo in os.listdir(os.path.join(path,f"{subcarpetas} {a}")):
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(path, f"{subcarpetas} {a}", archivo)
            nombre_archivo = archivo.split('_')[-1].split('.')[0][0:4] + '-' + archivo.split('_')[-1].split('.')[0][4:]
            datos_archivo_plano[nombre_archivo] = pd.read_csv(ruta_archivo, sep=';', encoding='ISO-8859-1', header=None, names=columnas)
            datos_archivo_plano[nombre_archivo]['VALOR_UNITARIO'] = datos_archivo_plano[nombre_archivo]['VALOR_UNITARIO'].replace(',','.', regex=True)
            datos_archivo_plano[nombre_archivo]['VALOR_UNITARIO'] = datos_archivo_plano[nombre_archivo]['VALOR_UNITARIO'].astype(float)


#### Como ayuda visual imprimimos los tipos de datos de cada columna de datos del DataFrame "datos_archivo_plano" del año 2017 del mes Febrero

In [ ]:
datos_archivo_plano['2017-02'].dtypes

##### Nos Aseguramos de que la Columna 4:'CEDULA' y la columna 20:'IDENTIFICACION_AFILIADO' este formateada en tipo texto con 10 caracteres y si no es asi ponga el "0" al inicio del numero, en todos los archivos planos que hemos cargado

In [ ]:
for key in datos_archivo_plano.keys():
    datos_archivo_plano[key]['CEDULA'] = datos_archivo_plano[key]['CEDULA'].astype(str).str.zfill(10)
    datos_archivo_plano[key]['IDENTIFICACION_AFILIADO'] = datos_archivo_plano[key]['IDENTIFICACION_AFILIADO'].astype(str).str.zfill(10)
    # print(key)

#### Realizamos el cotejamiento de cada una de las cedulas en el listado de trabajadores de la Empresa los Alamos, y lo verificamos contra la columan 20:'IDENTIFICACION_AFILIADO', la cual es el que acredita el derecho de atencion, utilizamos la comuna 'PERIODO' para limitar la busqueda a uno de los archivos planos previamente cargados; Y que las coincidencias que encuentre las separe en un DataFrame diferente para no alterar los datos originales puros; se imprime una ayuda visual para saber cuantas cedulas han sido procesadas ya que este paso puede demorar un poco

In [ ]:
dfAlamos = {}
for (i,p) in alamos.iterrows():
    key = p['PERIODO'].split('-')[0]+'-'+p['PERIODO'].split('-')[-1].zfill(2)
    df = datos_archivo_plano[key].copy()
    df2 = df[df['IDENTIFICACION_AFILIADO'].str.contains(p['CEDULA'])]
    if key not in dfAlamos:
        dfAlamos[key] = df2
    else:
        dfAlamos[key] = pd.concat([dfAlamos[key], df2])
    if i%100==0:
        print(f"{i} de {len(alamos)}", end=" | ")
print(f"{len(alamos)} de {len(alamos)}")


#### Imprimimos los años y mese de los archivos planos en lo que se hayan encontramos coincidencias

In [ ]:
print(dfAlamos.keys())

#### en la carpeta "periodos" dentro de este proyecto se almacenaran los archivos planos filtrados con unicamente las coincidencias que se encontraron en los pasos anteriroes

In [ ]:
for key in dfAlamos.keys():
    # print(os.path.join(path,'periodos', key.replace('-','') + '.xlsx'))
    dfAlamos[key].to_excel(os.path.join(path,'periodos', key.replace('-','') + '.xlsx'), index=False,header=columnas)
    # print(key)
print("Generados los Archivos Filtrados")

#### Una vez obtenido el DataFrame "dfAlamos" con los datos filtrados, en el DataFrame "alamos" procedemos a insertar los datos filtrados en pasos anteriores, una vez que se consigue una coincidencia de la 'CEDULA' CON 'IDENTIFICACION_AFILIADO', se crea temporalmente la columan TOTAL la cual contendrá el resultado de CANTIDAD * VALOR_UNITARIO del archivo plano, se agrupa la coindencia por la 'FECHA_ATENCION' con la sumatoria de 'TOTAL', se inserta las fechas obtenidos en la columna 'FECHA_PRESTACIONES_MEDICAS1' en formato texto separadas por ' ' y se inserta la sumatoria total de todas las fechas en la columna 'VALOR_TOTAL'

In [ ]:
for i, p in alamos.iterrows():
    periodo = p['PERIODO'].split('-')[0] + '-' + p['PERIODO'].split('-')[-1].zfill(2)
    df = dfAlamos.get(periodo)  # Buscar el DataFrame correspondiente a 'periodo'
    
    if df is not None:
        df2 = df[df['IDENTIFICACION_AFILIADO'].str.contains(p['CEDULA'], na=False)]
        
        if not df2.empty:
            df2['TOTAL'] = df2['CANTIDAD'] * df2['VALOR_UNITARIO']
            df2['TOTAL'] = df2['TOTAL'].astype(float)
            sumatoria = df2.groupby(['FECHA_ATENCION'])['TOTAL'].sum()
            alamos.at[i, 'FECHA_PRESTACIONES_MEDICAS1'] = ' '.join(sumatoria.keys().values)
            alamos.at[i, 'VALOR_TOTAL'] = str(sumatoria.sum()).replace('.', ',')
        # print(' '.join(sumatoria.keys().values),str(sumatoria.sum()).replace('.',','))
    # break

#### Para asegurar la integridad del excel original con el listado de los Alamos, guardamos todos lo realizado en un neuvoe excel con el nombre se se crea conveniente en este caso "alamos-201702-201812-procesado"

In [ ]:
alamos.to_excel(os.path.join(path,'alamos-201702-201812-procesado.xlsx'), index=False)